In [ ]:
import sys
sys.path.append("../src/")

In [ ]:
import s3fs

fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'})

In [ ]:
fs.ls("projet-slums-detection/Donnees/ADMINEXPRESS/guadeloupe/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGAF09UTM20_GLP")

In [ ]:
import geopandas as gpd

region_border = gpd.read_file("s3://projet-slums-detection/Donnees/ADMINEXPRESS/guadeloupe/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGAF09UTM20_GLP/REGION.shp")

In [ ]:
region_border.geometry.plot()

In [ ]:
from pathlib import Path
import re

def dep_from_train_path(path: str):
    """
    Get and return dep from train image path.

    Args:
        path (str): Path of train image.
    Return:
        str: Département.
    """
    file_name = Path(path).parent.parent.name
    pattern = r"^.*-(97[0-9]{1})"

    # Use re.match to find the match at the beginning of the string
    match = re.match(pattern, file_name)

    if match:
        dep = match.group(1)
        if dep not in [
            "971",
            "973",
            "972",
            "976",
            "974",
            "977",
            "978",
        ]:
            raise ValueError(f"Département {dep} not valid.")
    else:
        raise ValueError("Département not found in path.")
    return dep

In [ ]:
# Get a few satellite images
paths = [
    "/home/onyxia/work/train_data2-segmentation-PLEIADES-BDTOPO-segmentation-976-2022/images/ORT_976_2022_0504_8602_U38S_8Bits_0000.jp2",
    "/home/onyxia/work/train_data2-segmentation-PLEIADES-BDTOPO-segmentation-976-2022/images/ORT_976_2022_0505_8595_U38S_8Bits_0000.jp2",
    "/home/onyxia/work/train_data2-segmentation-PLEIADES-BDTOPO-segmentation-976-2022/images/ORT_976_2022_0505_8595_U38S_8Bits_0001.jp2",
]
dep_from_train_path(paths[0])

In [ ]:
from classes.data.satellite_image import SatelliteImage

In [ ]:
file_path = paths[0]
dep = dep_from_train_path(paths[0])

img = SatelliteImage.from_raster(file_path=file_path, dep=dep, date=None, n_bands=3)

In [ ]:
guadeloupe_border = gpd.read_file("s3://projet-slums-detection/Donnees/ADMINEXPRESS/guadeloupe/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGAF09UTM20_GLP/REGION.shp")
martinique_border = gpd.read_file("s3://projet-slums-detection/Donnees/ADMINEXPRESS/martinique/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGAF09UTM20_MTQ/REGION.shp")
guyane_border = gpd.read_file("s3://projet-slums-detection/Donnees/ADMINEXPRESS/guyane/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_UTM22RGFG95_GUF/REGION.shp")
mayotte_border = gpd.read_file("s3://projet-slums-detection/Donnees/ADMINEXPRESS/mayotte/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGM04UTM38S_MYT/REGION.shp")

region_borders = {
    "971": guadeloupe_border,
    "972": martinique_border,
    "973": guyane_border,
    "976": mayotte_border
}

In [ ]:
from shapely.geometry import box

In [ ]:
def is_within_borders(satellite_image: SatelliteImage, area_rate: float = 0.2):
    dep = satellite_image.dep
    try:
        border = region_borders[dep]
    except ValueError:
        raise ValueError(f"Département {dep} does not have registered borders.")
    bounds = satellite_image.bounds
    geom = box(*bounds)

    intersection_area = geom.intersection(border.geometry).area[0]
    return intersection_area / geom.area > area_rate

In [ ]:
is_within_borders(img)

In [ ]:
img.plot(bands_indices=[0, 1, 2])

In [ ]:
idx = 2

In [ ]:
file_path = paths[idx]
dep = dep_from_train_path(paths[idx])

img = SatelliteImage.from_raster(file_path=file_path, dep=dep, date=None, n_bands=3)

In [ ]:
is_within_borders(img)

In [ ]:
img.plot(bands_indices=[0, 1, 2])

In [ ]:
import os

In [ ]:
directory = "/home/onyxia/work/train_data2-segmentation-PLEIADES-BDTOPO-segmentation-976-2022/images/"

for idx in range(len(os.listdir(directory))):
    file_path = directory + os.listdir(directory)[idx]
    dep = dep_from_train_path(file_path)
    img = SatelliteImage.from_raster(file_path=file_path, dep=dep, date=None, n_bands=3)
    if not is_within_borders(img):
        print(idx)
        break

In [ ]:
img.plot(bands_indices=[0, 1, 2])